In [41]:
import duckdb
import pandas as pd
import json

conn = duckdb.connect()

query = """
        SELECT DISTINCT name
        , id
        , oracle_id
        , edhrec_rank
        , mana_cost
        , cmc
        , CASE 
            WHEN LOWER(type_line) LIKE '%creature%' THEN 'Creature' 
            WHEN LOWER(type_line) LIKE '%artifact%' THEN 'Artifact' 
            WHEN LOWER(type_line) LIKE '%enchantment%' THEN 'Enchantment' 
            WHEN LOWER(type_line) LIKE '%planeswalker%' THEN 'Planeswalker' 
            WHEN LOWER(type_line) LIKE '%battle%' THEN 'Battle' 
            WHEN LOWER(type_line) LIKE '%land%' THEN 'Land' 
            WHEN LOWER(type_line) LIKE '%instant%' THEN 'Instant' 
            WHEN LOWER(type_line) LIKE '%sorcery%' THEN 'Sorcery' 
            ELSE type_line END as primary_type
        , concat('[',replace(split_part(trim(type_line), '— ', 2),' ',', '),']') as secondary_type
        , regexp_replace(regexp_replace(oracle_text, '\([^)]*\)', '', 'g'),'\s+', ' ', 'g') AS oracle_text
        , color_identity
        , keywords
        , CASE WHEN power LIKE '%*%' THEN NULL ELSE power END as power
        , CASE WHEN toughness LIKE '%*%' THEN NULL ELSE toughness END as toughness
        , scryfall_uri
        , CASE 
            WHEN prices.usd IS NOT NULL 
            THEN CAST(prices.usd AS DECIMAL(10,2))
            ELSE NULL END as usd_price
        , image_uris.png as png_url
    FROM read_json_auto('data/raw/oracle-cards.json')
    WHERE layout = 'normal'
        AND legalities.commander = 'legal'
        AND (LOWER(type_line) LIKE '%creature%' OR LOWER(oracle_text) LIKE '%can be your commander%')
"""

conn.execute(f"COPY ({query}) TO 'data/raw/all_creatures_clean.csv' (HEADER, DELIMITER ',')")

conn.close()